In [12]:
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/



fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.
mkdir: cannot create directory '/kaggle/working/gemma/': File exists
mv: cannot stat '/kaggle/working/gemma_pytorch/gemma/*': No such file or directory


In [13]:
import sys 
sys.path.append("/kaggle/working/gemma_pytorch/") 
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch
import json
import random
import pandas as pd
import re

# Load the model
VARIANT = "7b-it-quant" 
MACHINE_TYPE = "cuda" 
weights_dir = '/kaggle/input/gemma/pytorch/7b-it-quant/2' 

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")
model_config.quant = "quant" in VARIANT

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()



# Use the model

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn>\n"

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="What is a good place for travel in the US?"
    )
    + MODEL_CHAT_TEMPLATE.format(prompt="California.")
    + USER_CHAT_TEMPLATE.format(prompt="What can I do in California?")
    + "<start_of_turn>model\n"
)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)


'California offers a diverse range of experiences, including:\n\n**City life:**\n- San Francisco: Golden Gate Bridge, Alcatraz Island, Chinatown, Haight-Ashbury neighborhood\n- Los Angeles: Griffith Observatory, Hollywood Walk of Fame, Santa Monica Pier, Disneyland Park\n- San Diego: Coronado Island, Gaslamp Quarter, SeaWorld San Diego, La Jolla Cove\n\n**Natural beauty:**\n- Yosemite National Park: El Capitan, Half Dome, Glacier Point\n- Redwood National Park:'

In [14]:
#     def create_prompt_shot(self,sample):
#         # Example prompt
#         prompt = """Read the following table and answer the related question based on the table.\n\n"""
#         prompt += 'TABLE:\n'
#         prompt += 'Column Name: '
#         prompt += ','.join(
#             [f'"{cc}"' for cc in sample['table']['cols']]
#         ) + '\n'
#         i=1
#         for row in sample['table']['rows']:
#             prompt += 'Row '+ str(i)+ ': '
#             i+=1
#             prompt += ','.join(
#                 [f'"{rr}"' for rr in row]
#             ) + '\n'
#         prompt += '\n'
#         prompt += 'QUESTION: ' + sample['question'] + '\n' 
#         prompt += 'Give the column name and row number from the table to answer the question.' + '\n'
#         prompt += "The column name is '" + sample['label_col'] + "' and the row number is " + sample['label_row'] + '\n'
#         prompt += '\n'

#         return prompt

In [15]:
class PromptGenerator(object):
    def __init__(self):
        """
        Setup generation parameters for Gemma.
        """
        self.output_len = 25
        self.temperature = 0.0
        self.top_p = 1.0
        
    def create_prompt(self, sample):       
        
        prompt = """Read the following table and answer the related question based on the table.

TABLE:
Column Name: "Date","Location","Opponenent","Result","Match type"
Row 1: "29 March 2000","Debrecen","Poland","0-0 (draw)","friendly"
Row 2: "16 August 2000","Budapest","Austria","1-1 (draw)","friendly"
Row 3: "3 September 2000","Budapest","Italy","2-2 (draw)","WC -qualifier"
Row 4: "15 August 2001","Budapest","Germany","2-5 (defeat)","friendly"
Row 5: "1 September 2001","Tbilisi","Georgia","1-3 (defeat)","WC-qualifier"
Row 6: "5 September 2001","Budapest","Romania","0-2 (defeat)","WC-qualifier"
Row 7: "6 October 2001","Parma","Italy","0-0 (draw)","WC-qualifier"
Row 8: "14 November 2001","Budapest","Macedonia","5-0 (win)","friendly"
Row 9: "12 February 2002","Larnaca","Czech Rep.","0-2 (defeat)","friendly"
Row 10: "13 February 2002","Limassol","Switzerland","1-2 (defeat)","friendly"
Row 11: "8 May 2002","Pécs","Croatia","0-2 (defeat)","friendly"
Row 12: "21 August 2002","Budapest","Spain","1-1 (draw)","friendly"
Row 13: "7 September 2002","Reykjavík","Iceland","2-0 (win)","friendly"
Row 14: "12 October 2002","Stockholm","Sweden","1-1 (draw)","EC-qualifier"
Row 15: "16 October 2002","Budapest","San Marino","3-0 (win)","EC -qualifier"
Row 16: "20 November 2002","Budapest","Moldova","1-1 (draw)","friendly"
Row 17: "12 February 2003","Larnaca","Bulgaria","0-1 (defeat)","friendly"
Row 18: "29 March 2003","Chorzów","Poland","0-0 (draw)","EC -qualifier"
Row 19: "2 April 2003","Budapest","Sweden","1-2 (defeat)","EC -qualifier"
Row 20: "30 April 2003","Budapest","Luxembourg","5-1 (win)","friendly"
Row 21: "19 February 2004","Limassol","Latvia","2-1 (win)","friendly"
Row 22: "21 February 2004","Limassol","Romania","0-3 (defeat)","friendly"
Row 23: "9 February 2011","Dubai","Azerbaijan","2-0 (win)","friendly"
Row 24: "29 March 2011","Amsterdam","Netherlands","3-5 (defeat)","EC -qualifier"
Row 25: "3 June 2011","Luxembourg","Luxembourg","1-0 (win)","friendly"

QUESTION: When was the opponent Poland and the match type EC -qualifier?
Give the column name and row number from the table to answer the question.
The column name is 'Date' and the row number is 18

Read the following table and answer the related question based on the table.

TABLE:
Column Name: "Average Ranking","Competitive Finish","Couple","Number Of Dances","Total Score","Average"
Row 1: "1","1","Bridie & Craig","15","509","35.9"
Row 2: "2","3","David & Karina","12","360","30.0"
Row 3: "3","4","Patti & Sandro","10","295","29.5"
Row 4: "4","2","Anh & Luda","15","421","27.0"
Row 5: "5","9","Corinne & Csaba","3","77","25.7"
Row 6: "6","5","Mark & Linda","8","204","25.5"
Row 7: "7","8","Elka & Michael","4","100","25.0"
Row 8: "8","6","James & Olya","7","169","24.1"
Row 9: "9","7","Jessica & Serghei","5","120","24.0"

QUESTION: What is the total score when 7 is the average ranking?
Give the column name and row number from the table to answer the question.
The column name is 'Total Score' and the row number is 7"""
        
        # Example prompt        
        prompt += '\n'
        prompt += """Read the following table and answer the related question based on the table.\n\n"""
        prompt += 'TABLE:\n'
        prompt += 'Column Name: '
        prompt += ','.join(
            [f'"{cc}"' for cc in sample['table']['cols']]
        ) + '\n'
        i=1
        for row in sample['table']['rows']:
            prompt += 'Row '+ str(i)+ ': '
            i+=1
            prompt += ','.join(
                [f'"{rr}"' for rr in row]
            ) + '\n'
            if len(prompt)>7800:
                break
        prompt += '\n'
        prompt += 'QUESTION: ' + sample['question'] +'\n'
        prompt += 'Give the column name and row number from the table to answer the question.' + '\n'
        print('prompt length is ')
        print(len(prompt))

        return prompt
    
    def post_process(self, gen_text):
        gen_text = gen_text.split("\n")[0]
        pattern = r"The column name is \'(.+?)\' and the row number is (\d+)"

        match = re.search(pattern, gen_text)

        if match:
                column_name = match.group(1)
                row_number = int(match.group(2))
                column_name = column_name.replace("'", "")
                row_number-=1
                return_list = []
                return_pair = row_number, column_name
                return_list.append(return_pair)
                return return_list
        else:
                return_list = []
                return return_list 

    

In [16]:
# sample_shot = {
#     "question":"When was the opponent Poland and the match type EC -qualifier?",
#     "table":{
#         "cols":["Date","Location","Opponenent","Result","Match type"],
#         "rows":[
#             ["29 March 2000","Debrecen","Poland","0-0 (draw)","friendly"],
#             ["16 August 2000","Budapest","Austria","1-1 (draw)","friendly"],
#             ["3 September 2000","Budapest","Italy","2-2 (draw)","WC -qualifier"],
#             ["15 August 2001","Budapest","Germany","2-5 (defeat)","friendly"],
#             ["1 September 2001","Tbilisi","Georgia","1-3 (defeat)","WC-qualifier"],
#             ["5 September 2001","Budapest","Romania","0-2 (defeat)","WC-qualifier"],
#             ["6 October 2001","Parma","Italy","0-0 (draw)","WC-qualifier"],
#             ["14 November 2001","Budapest","Macedonia","5-0 (win)","friendly"],
#             ["12 February 2002","Larnaca","Czech Rep.","0-2 (defeat)","friendly"],
#             ["13 February 2002","Limassol","Switzerland","1-2 (defeat)","friendly"],
#             ["8 May 2002","P\u00e9cs","Croatia","0-2 (defeat)","friendly"],
#             ["21 August 2002","Budapest","Spain","1-1 (draw)","friendly"],
#             ["7 September 2002","Reykjav\u00edk","Iceland","2-0 (win)","friendly"],
#             ["12 October 2002","Stockholm","Sweden","1-1 (draw)","EC-qualifier"],
#             ["16 October 2002","Budapest","San Marino","3-0 (win)","EC -qualifier"],
#             ["20 November 2002","Budapest","Moldova","1-1 (draw)","friendly"],
#             ["12 February 2003","Larnaca","Bulgaria","0-1 (defeat)","friendly"],
#             ["29 March 2003","Chorz\u00f3w","Poland","0-0 (draw)","EC -qualifier"],
#             ["2 April 2003","Budapest","Sweden","1-2 (defeat)","EC -qualifier"],
#             ["30 April 2003","Budapest","Luxembourg","5-1 (win)","friendly"],
#             ["19 February 2004","Limassol","Latvia","2-1 (win)","friendly"],
#             ["21 February 2004","Limassol","Romania","0-3 (defeat)","friendly"],
#             ["9 February 2011","Dubai","Azerbaijan","2-0 (win)","friendly"],
#             ["29 March 2011","Amsterdam","Netherlands","3-5 (defeat)","EC -qualifier"],
#             ["3 June 2011","Luxembourg","Luxembourg","1-0 (win)","friendly"]
#         ],
#         "types":["text","text","text","text","text"],
#         "caption":"National team matches",
#     },
#     "label_col":"Date",
#     "label_row":"18"
# }
# sample_shot

# sample_shot2 = {
#     "question":"What is the total score when 7 is the average ranking?",
#     "table":{
#         "cols":["Average Ranking","Competitive Finish","Couple","Number Of Dances","Total Score","Average"],
#         "rows":[
#             ["1","1","Bridie & Craig","15","509","35.9"],
#             ["2","3","David & Karina","12","360","30.0"],
#             ["3","4","Patti & Sandro","10","295","29.5"],
#             ["4","2","Anh & Luda","15","421","27.0"],
#             ["5","9","Corinne & Csaba","3","77","25.7"],
#             ["6","5","Mark & Linda","8","204","25.5"],
#             ["7","8","Elka & Michael","4","100","25.0"],
#             ["8","6","James & Olya","7","169","24.1"],
#             ["9","7","Jessica & Serghei","5","120","24.0"]
#         ],
#         "types":["real","real","text","real","real","text"],
#         "caption":"Average Chart"
#     },
#     "label_col":"Total Score",
#     "label_row":"7"
# }
# sample_shot2

# # sample_shot3 = {
# #     "question":"What was the largest crowd size at a South Melbourne home game?",
# #     "table":{
# #         "cols":["Home team","Home team score","Away team","Away team score","Venue","Crowd","Date"],
# #         "rows":[
# #             ["Melbourne","15.15 (105)","St Kilda","5.13 (43)","MCG","11,000","7 June 1947"],
# #             ["Hawthorn","10.15 (75)","North Melbourne","10.11 (71)","Glenferrie Oval","12,000","7 June 1947"],
# #             ["Carlton","16.21 (117)","Geelong","7.10 (52)","Princes Park","16,000","7 June 1947"],
# #             ["South Melbourne","15.9 (99)","Richmond","15.13 (103)","Lake Oval","24,000","7 June 1947"],
# #             ["Footscray","12.11 (83)","Fitzroy","7.7 (49)","Western Oval","20,000","7 June 1947"],
# #             ["Essendon","9.14 (68)","Collingwood","11.16 (82)","Windy Hill","22,000","7 June 1947"]
# #         ],
# #         "types":["text","text","text","text","text","real","text"],
# #         "caption":"Round 8"
# #     },
# #     "label_col":"Crowd",
# #     "label_row":"4"
# # }


In [17]:
# prompt_generator = PromptGenerator()
# prompt_shot = prompt_generator.create_prompt_shot(sample_shot)
# prompt_shot2 = prompt_generator.create_prompt_shot(sample_shot2)
# promp = prompt_shot + prompt_shot2
# print(promp)
# # # prompt_shot3 = prompt_generator.create_prompt_shot(sample_shot3)
# # prompt_act = prompt_generator.create_prompt(samp)
# # prompt = prompt_shot + prompt_shot2 + prompt_act
# # print(prompt)
# # answer = model.generate(
# #     USER_CHAT_TEMPLATE.format(prompt=prompt),
# #     device=device,
# #     output_len=prompt_generator.output_len,
# #     temperature=prompt_generator.temperature,
# #     top_p=prompt_generator.top_p,
# # )


In [18]:
# print(answer)

In [19]:
data = []
with open('/kaggle/input/a2-test/A2_test.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

samples = data
# samples = random.sample(data, 100)


In [ ]:
# Generate the answers
prompt_generator = PromptGenerator()
# prompt_shot = prompt_generator.create_prompt_shot(sample_shot)
# prompt_shot2 = prompt_generator.create_prompt_shot(sample_shot2)
# prompt_shot3 = prompt_generator.create_prompt_shot(sample_shot3)
# col_answers = []
# row_answers = []
cell_answers = []
num=1
index = 0
for sample in samples:
    print("Sample number is "+ str(num))
    print("Question is "+ sample['question'])
    print(len(sample['table']['rows']))
    cur_sample = {'question': sample['question'], 'table': sample['table']}
    prompt = prompt_generator.create_prompt(cur_sample)
    if num==1:
        print(prompt)
#     prompt = prompt_shot + prompt_shot2 + prompt_act
    answer = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=prompt_generator.output_len,
        temperature=prompt_generator.temperature,
        top_p=prompt_generator.top_p,
    )
    answer_list = prompt_generator.post_process(answer)
    print(answer)
    print(answer_list)
    cell_answers.append(answer_list)
#     print("Column should be "+ sample['label_col'][0])
#     print("Row should be "+ ', '.join([str(rr) for rr in sample['label_row']]))
    print("Cells should be ")
    print(sample['label_cell'])
    print("===================================================================")
#     col_answers.append(answer_pair[0])
#     row_answers.append(answer_pair[1])
    num+=1
#     torch.cuda.empty_cache()
#     del cur_sample
#     del prompt_act
#     del prompt
#     del answer
#     del answer_list

correct_col = 0
correct_row = 0
correct_cell = 0

for sample, answer in zip(samples, cell_answers):
    if len(sample['label_cell']) == len(answer):
        if sample['label_cell'][0][0]== answer[0][0] and sample['label_cell'][0][1]==answer[0][1]:
            correct_cell += 1
        if sample['label_cell'][0][0]== answer[0][0]:
            correct_row += 1
        if sample['label_cell'][0][1]== answer[0][1]:
            correct_col += 1
    else:
        if len(answer)>0 and sample['label_cell'][0][1]== answer[0][1]:
            correct_col += 1
cell_acc = correct_cell / len(samples)
col_acc = correct_col / len(samples)
row_acc = correct_row / len(samples)

In [21]:
print(cell_acc)


0.7


In [22]:
print(col_acc)
print(row_acc)

0.81
0.79
